
-----------------------------------------------------
### Notebook Documentation.
#### 1. TFM
---------------------------------
Different architectures / parameters.


In [ ]:
# Google Drive stuff
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

- libraries

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
%tensorflow_version 2.x
# batch ingestion of pics without pickle
from tensorflow.keras.preprocessing import image_dataset_from_directory

# nns
from tensorflow.keras.applications import VGG16, VGG19,Xception, InceptionResNetV2, DenseNet121, ResNet50 

from tensorflow.keras import Model
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras import layers # for data augmentation
#from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, MaxPooling2D, Flatten # Input, Conv2D, 
#from tensorflow.keras.layers import Layer

# optimization
from tensorflow.keras.optimizers import Adam #, SGD
from tensorflow.keras.losses import SparseCategoricalCrossentropy, categorical_crossentropy
from tensorflow.keras.callbacks import EarlyStopping

# metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
#from sklearn.metrics import plot_precision_recall_curve
from itertools import chain # to flatten the real labels array from validation set
import json # to save in a file metrics
#from datetime import datetime # to name results

# viz & arrays
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


# my functions
from my_functions_trainvaltest import (vgg19_vgg16,
                                       generic_last_2layers,
                                       plotting_model,
                                       model_evaluation,
                                       classification_report_pic,
                                       confusion_matrix_report
                                      )

- paths

In [ ]:
# paths i'll use
base_folder = "/content/drive/My Drive/2-Estudios/viu-master_ai/tfm-deep_vision/"
input_folder = base_folder+"input/" #for pickling data
input = input_folder+"House_Room_Dataset-5_rooms" # for requesting directly pics
src_folder =  base_folder+"src/"
output_folder = base_folder + "/output/"

# insert the directory for using my functions in this notebook
import sys
sys.path.insert(0,"/content/drive/My Drive/2-Estudios/viu-master_ai/tfm-deep_vision/src/")

# getting pics from folder instead of pickling them

In previous versions I used a pickled dataset to work with. That was quite a mess, because it loads the full dataset in memory, so sometimes the notebook crashed. Now I batch process the pictures with this keras method => https://keras.io/examples/vision/image_classification_from_scratch/
- *image_dataset_from_directory* has no train-test-val. it has just train-val.

In [ ]:
image_size = (128,128)
batch_size = 128 # 32

In [ ]:
# train set
# https://www.tensorflow.org/tutorials/images/classification
train_ds = image_dataset_from_directory(
    input,
    class_names=["Bedroom","Bathroom","Dinning","Livingroom","Kitchen"],
    # labels='inferred',
    # label_mode='categorical',
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size= image_size,
    batch_size= batch_size,
    color_mode='rgb'
)

In [ ]:
# validation set (use the same seed and split)
val_ds = image_dataset_from_directory( # utilizar el de validación como test
    input,
    # labels='inferred',
    # label_mode='categorical',
    class_names=["Bedroom","Bathroom","Dinning","Livingroom","Kitchen"],
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=image_size,
    batch_size=batch_size,
    color_mode='rgb'
)

In [ ]:
class_names = train_ds.class_names
class_names

# data augmentation

In [ ]:
# https://www.tensorflow.org/guide/keras/preprocessing_layers
data_augmentation = Sequential(  
  [
    layers.RandomFlip("horizontal",input_shape=(image_size[0],image_size[1],3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.005),
    #random_invert(0.2),  # color inversion
    layers.RandomContrast(0.1),
    #Contrast(),
    #layers.RandomCrop(image_size[0],image_size[1]), 
    layers.RandomWidth(0.05, interpolation="gaussian"),
    layers.RandomHeight(0.05),
    layers.RandomTranslation((-0.2,0.3),(-0.2,0.3), fill_mode='constant',interpolation='bilinear', seed=None, fill_value=0.0),
  ]
)

plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
  for i in range(9):
    augmented_images = data_augmentation(images)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_images[0].numpy().astype("uint8"))
    plt.axis("off")

# Models with/without data augmentation

In [ ]:
base_model_vgg16 = VGG16(include_top=False, weights='imagenet', input_shape=(128, 128, 3), classes = 5, classifier_activation='softmax')
base_model_vgg19 = VGG19(include_top=False, weights='imagenet', input_shape=(128, 128, 3), classes = 5, classifier_activation='softmax')
base_model_xception    =  Xception(include_top=False, weights='imagenet', input_shape=(128, 128, 3), classes = 5, classifier_activation='softmax')
base_model_inception   = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=(128, 128, 3), classes = 5, classifier_activation='softmax')
base_model_densenet121 = DenseNet121(include_top=False, weights='imagenet', input_shape=(128, 128, 3), classes = 5) # has no classifier activation args
base_model_resnet50    = ResNet50(include_top=False, weights='imagenet', input_shape=(128, 128, 3), classes = 5) # has no classifier activation args

models_dict = {
    # vgg16
    "vgg16_NOdataAug_NOdropout"      : vgg19_vgg16(data_augmentation=None, 
                                                   base_model= base_model_vgg16, dropout_layers=False, 
                                                   dropout_position= None, 
                                                   dropout_percent=None, 
                                                   num_classes=5),
    
    "vgg16_dataAug_dropoutFirst02"   : vgg19_vgg16(data_augmentation=data_augmentation,  
                                                   base_model= base_model_vgg16, dropout_layers=True,  
                                                   dropout_position= "first",  
                                                   dropout_percent=0.2,  
                                                   num_classes=5),
    
    "vgg16_NOdataAug_dropoutFirst02" : vgg19_vgg16(data_augmentation=None,               
                                                   base_model= base_model_vgg16, dropout_layers=True,  
                                                   dropout_position= "first",  
                                                   dropout_percent=0.2,  
                                                   num_classes=5),
    
    "vgg16_dataAug_dropoutMiddle02"  : vgg19_vgg16(data_augmentation=data_augmentation,  
                                                   base_model= base_model_vgg16, 
                                                   dropout_layers=True,  
                                                   dropout_position= "middle", 
                                                   dropout_percent=0.2,  
                                                   num_classes=5),
    
    "vgg16_NOdataAug_dropoutMiddle02": vgg19_vgg16(data_augmentation=None,               
                                                   base_model= base_model_vgg16, 
                                                   dropout_layers=True,  
                                                   dropout_position= "middle", 
                                                   dropout_percent=0.2,  
                                                   num_classes=5),
    
    "vgg16_dataAug_NOdropout"        : vgg19_vgg16(data_augmentation=data_augmentation,  
                                                   base_model= base_model_vgg16, 
                                                   dropout_layers=False, 
                                                   dropout_position= None,     
                                                   dropout_percent=None, 
                                                   num_classes=5),   
    # vgg19
    "vgg19_NOdataAug_NOdropout"      : vgg19_vgg16(data_augmentation=None,               
                                                   base_model= base_model_vgg19, 
                                                   dropout_layers=False, 
                                                   dropout_position= None,     
                                                   dropout_percent=None, 
                                                   num_classes=5),
    
    "vgg19_dataAug_dropoutFirst02"   : vgg19_vgg16(data_augmentation=data_augmentation,  
                                                   base_model= base_model_vgg19, 
                                                   dropout_layers=True,  
                                                   dropout_position= "first",  
                                                   dropout_percent=0.2,  
                                                   num_classes=5),
    
    "vgg19_NOdataAug_dropoutFirst02" : vgg19_vgg16(data_augmentation=None,               
                                                   base_model= base_model_vgg19, 
                                                   dropout_layers=True,  
                                                   dropout_position= "first",  
                                                   dropout_percent=0.2,  
                                                   num_classes=5),
    
    "vgg19_dataAug_dropoutMiddle02"  : vgg19_vgg16(data_augmentation=data_augmentation,  
                                                   base_model= base_model_vgg19, 
                                                   dropout_layers=True,  
                                                   dropout_position= "middle", 
                                                   dropout_percent=0.2,  
                                                   num_classes=5),
    
    "vgg19_NOdataAug_dropoutMiddle02": vgg19_vgg16(data_augmentation=None,               
                                                   base_model= base_model_vgg19, 
                                                   dropout_layers=True,  
                                                   dropout_position= "middle", 
                                                   dropout_percent=0.2,  
                                                   num_classes=5),
    
    "vgg19_dataAug_NOdropout"        : vgg19_vgg16(data_augmentation=data_augmentation,  
                                                   base_model= base_model_vgg19, 
                                                   dropout_layers=False, 
                                                   dropout_position= None,     
                                                   dropout_percent=None, 
                                                   num_classes=5),    
    # xception
    "xception_NOdataAug_NOdropout"     : generic_last_2layers(data_augmentation=None,  
                                                              nn=base_model_xception, 
                                                              neurons_final_layer=5, 
                                                              dropout_layers=False, 
                                                              dropout_position=None,     
                                                              dropout_percent = None),
    
    "xception_dataAug_NOdropout"       : generic_last_2layers(data_augmentation,       
                                                              nn=base_model_xception, 
                                                              neurons_final_layer=5, 
                                                              dropout_layers=False, 
                                                              dropout_position=None,     
                                                              dropout_percent = None),
    
    "xception_dataAug_dropoutFirst02"  : generic_last_2layers(data_augmentation,       
                                                              nn=base_model_xception, 
                                                              neurons_final_layer=5, 
                                                              dropout_layers=True,  
                                                              dropout_position="first",  
                                                              dropout_percent = 0.2),
    
    "xception_NOdataAug_dropoutFirst02": generic_last_2layers(data_augmentation= None, 
                                                              nn=base_model_xception, 
                                                              neurons_final_layer=5, 
                                                              dropout_layers=True,  
                                                              dropout_position="first",  
                                                              dropout_percent = 0.2),
    
    "xception_dataAug_dropoutMid02"    : generic_last_2layers(data_augmentation,       
                                                              nn=base_model_xception, 
                                                              neurons_final_layer=5, 
                                                              dropout_layers=True,  
                                                              dropout_position="middle", 
                                                              dropout_percent = 0.2),
    
    "xception_NOdataAug_dropoutMid02"  : generic_last_2layers(data_augmentation= None, 
                                                              nn=base_model_xception, 
                                                              neurons_final_layer=5, 
                                                              dropout_layers=True,  
                                                              dropout_position="middle", 
                                                              dropout_percent = 0.2),    
    # # inception resnet v2
    "InceptionResNetV2_NOdataAug_NOdropout"     : generic_last_2layers(data_augmentation=None,  
                                                                       nn=base_model_inception, 
                                                                       neurons_final_layer=5, 
                                                                       dropout_layers=False, 
                                                                       dropout_position=None,     
                                                                       dropout_percent = None),
    
    "InceptionResNetV2_dataAug_NOdropout"       : generic_last_2layers(data_augmentation,       
                                                                       nn=base_model_inception, 
                                                                       neurons_final_layer=5, 
                                                                       dropout_layers=False, 
                                                                       dropout_position=None,     
                                                                       dropout_percent = None),
    
    "InceptionResNetV2_dataAug_dropoutFirst02"  : generic_last_2layers(data_augmentation,       
                                                                       nn=base_model_inception, 
                                                                       neurons_final_layer=5, 
                                                                       dropout_layers=True,  
                                                                       dropout_position="first",  
                                                                       dropout_percent = 0.2),
    
    "InceptionResNetV2_NOdataAug_dropoutFirst02": generic_last_2layers(data_augmentation= None, 
                                                                       nn=base_model_inception, 
                                                                       neurons_final_layer=5, 
                                                                       dropout_layers=True,  
                                                                       dropout_position="first",  
                                                                       dropout_percent = 0.2),
    
    "InceptionResNetV2_dataAug_dropoutMid02"    : generic_last_2layers(data_augmentation,       
                                                                       nn=base_model_inception, 
                                                                       neurons_final_layer=5, 
                                                                       dropout_layers=True,  
                                                                       dropout_position="middle", 
                                                                       dropout_percent = 0.2),
    
    "InceptionResNetV2_NOdataAug_dropoutMid02"  : generic_last_2layers(data_augmentation= None, 
                                                                       nn=base_model_inception, 
                                                                       neurons_final_layer=5, 
                                                                       dropout_layers=True, 
                                                                       dropout_position="middle", 
                                                                       dropout_percent = 0.2),
    # densenet121
    "densenet121_NOdataAug_NOdropout"     : generic_last_2layers(data_augmentation=None,  
                                                                 nn=base_model_densenet121, 
                                                                 neurons_final_layer=5, 
                                                                 dropout_layers=False, 
                                                                 dropout_position=None,     
                                                                 dropout_percent = None),
    
    "densenet121_dataAug_NOdropout"       : generic_last_2layers(data_augmentation,       
                                                                 nn=base_model_densenet121, 
                                                                 neurons_final_layer=5, 
                                                                 dropout_layers=False, 
                                                                 dropout_position=None,     
                                                                 dropout_percent = None),
    
    "densenet121_dataAug_dropoutFirst02"  : generic_last_2layers(data_augmentation,       
                                                                 nn=base_model_densenet121, 
                                                                 neurons_final_layer=5, 
                                                                 dropout_layers=True,  
                                                                 dropout_position="first",  
                                                                 dropout_percent = 0.2),
    
    "densenet121_NOdataAug_dropoutFirst02": generic_last_2layers(data_augmentation= None, 
                                                                 nn=base_model_densenet121, 
                                                                 neurons_final_layer=5, 
                                                                 dropout_layers=True,  
                                                                 dropout_position="first",  
                                                                 dropout_percent = 0.2),
    
    "densenet121_dataAug_dropoutMid02"    : generic_last_2layers(data_augmentation,       
                                                                 nn=base_model_densenet121, 
                                                                 neurons_final_layer=5, 
                                                                 dropout_layers=True,  
                                                                 dropout_position="middle", 
                                                                 dropout_percent = 0.2),
    
    "densenet121_NOdataAug_dropoutMid02"  : generic_last_2layers(data_augmentation= None, 
                                                                 nn=base_model_densenet121, 
                                                                 neurons_final_layer=5, 
                                                                 dropout_layers=True,  
                                                                 dropout_position="middle",
                                                                 dropout_percent = 0.2),
    # resnet50
    "resnet50_NOdataAug_NOdropout"     : generic_last_2layers(data_augmentation=None,  
                                                              nn=base_model_resnet50,
                                                              neurons_final_layer=5, 
                                                              dropout_layers=False, 
                                                              dropout_position=None,     
                                                              dropout_percent = None),
    
    "resnet50_dataAug_NOdropout"       : generic_last_2layers(data_augmentation,       
                                                              nn=base_model_resnet50, 
                                                              neurons_final_layer=5, 
                                                              dropout_layers=False, 
                                                              dropout_position=None,     
                                                              dropout_percent = None),
    
    "resnet50_dataAug_dropoutFirst02"  : generic_last_2layers(data_augmentation,       
                                                              nn=base_model_resnet50, 
                                                              neurons_final_layer=5, 
                                                              dropout_layers=True,  
                                                              dropout_position="first", 
                                                              dropout_percent = 0.2),
    
    "resnet50_NOdataAug_dropoutFirst02": generic_last_2layers(data_augmentation= None, 
                                                              nn=base_model_resnet50, 
                                                              neurons_final_layer=5, 
                                                              dropout_layers=True,  
                                                              dropout_position="first",  
                                                              dropout_percent = 0.2),
    
    "resnet50_dataAug_dropoutMid02"    : generic_last_2layers(data_augmentation,       
                                                              nn=base_model_resnet50, 
                                                              neurons_final_layer=5, 
                                                              dropout_layers=True,  
                                                              dropout_position="middle",
                                                              dropout_percent = 0.2),
    
    "resnet50_NOdataAug_dropoutMid02"  : generic_last_2layers(data_augmentation= None, 
                                                              nn=base_model_resnet50, 
                                                              neurons_final_layer=5, 
                                                              dropout_layers=True,  
                                                              dropout_position="middle", 
                                                              dropout_percent = 0.2),
}

In [ ]:
callbacks = [
    EarlyStopping(
        # Stop training when the metric is no longer improving
        monitor="val_accuracy", mode="max", #  "val_loss",
        # "no longer improving" being defined as "no better than 1e-4 less"
        min_delta=1e-4,
        # "no longer improving" being further defined as "for at least 10 epochs"
        patience=10,
        verbose=1
    )
]

epochs = 300
for m in models_dict:
  print("\n", m)
  models_dict[m].summary()

  models_dict[m].compile( optimizer = "adam", 
                loss=SparseCategoricalCrossentropy(from_logits=True) ,#'categorical_crossentropy', , 
                metrics=['accuracy'] # "recall"
                )

  history = models_dict[m].fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs,
        callbacks = callbacks # <=== REMOVE CALLBACK for full results
        )
  
  number_of_epochs_it_ran = len(history.history['loss']) 
  print("run epochs: ",number_of_epochs_it_ran)
  name = m
  models_dict[m].save(output_folder+name+".h5")

  # saving model accuracy/loss graph
  plotting_model(history,number_of_epochs_it_ran, name, output_folder+name+"-loss_accuracy.png") 
  
  # saving model metrics to json
  evaluation = models_dict[m].evaluate(val_ds, batch_size=batch_size, return_dict=True)
  model_evaluation(evaluation, output_folder, name)

  # get inferences
  y_pred_float = models_dict[m].predict(val_ds)
  y_pred = np.argmax(y_pred_float, axis=1) 

  # classification and confusion matrix reports
  classification_report_pic(y_pred, validation_ds,  class_names, output_folder, name)
  confusion_matrix_report(y_pred, validation_ds, class_names, output_folder, name)

- load a trained model

In [ ]:
# error above for misplacing y_pred and val_ds in function args
densenet121 = load_model(output_folder+"densenet121_dataAug_dropoutFirst02.h5")
evaluation  = densenet121.evaluate(val_ds, batch_size=batch_size, return_dict=True)

In [ ]:
# saving model metrics to json
model_evaluation(evaluation, output_folder, "densenet121_dataAug_dropoutFirst02.h5")

In [ ]:
# get inferences
y_pred_float = models_dict[m].predict(val_ds)
y_pred = np.argmax(y_pred_float, axis=1) 

In [ ]:
# classification and confusion matrix reports
classification_report_pic(y_pred, val_ds,  class_names, output_folder, "densenet121_dataAug_dropoutFirst02.h5")
confusion_matrix_report(y_pred, val_ds, class_names, output_folder, "densenet121_dataAug_dropoutFirst02.h5")